<a href="https://colab.research.google.com/github/probml/pyprobml/blob/master/book1/intro/tf_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to tensorflow 2.0

We show some simple examples of how to use TF 2.0 via the keras interface. 

**Make sure you select 'GPU' from the 'Runtime' tab at the top of this page.**

For more details, see 

* [The official TF tutorials](https://www.tensorflow.org/beta/tutorials)
* ["Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow v2"](https://github.com/ageron/handson-ml2), an excellent book by Aurelion Geron.
* ["Deep learning with Python"](https://github.com/fchollet/deep-learning-with-python-notebooks), a short book by Francois Chollet on Keras.



In [1]:
# Standard Python libraries
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio

from IPython import display

import sklearn

import seaborn as sns;
sns.set(style="ticks", color_codes=True)

import pandas as pd
pd.set_option('precision', 2) # 2 decimal places
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 100) # wide windows



In [2]:
# Tensorflow 2.0
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras

print("tf version {}".format(tf.__version__))
if tf.test.is_gpu_available():
    print(tf.test.gpu_device_name())
else:
    print("TF cannot find GPU")

tf version 2.4.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
/device:GPU:0


# Make an MLP 

See examples at https://github.com/probml/pyprobml/tree/master/book1/mlp

# Tensorflow datasets (TFDS)

Tensorflow has an easy way to load data, and convert it into a stream of miniatches, as we show below.


The functionality similar functionality to PyTorch DataLoader, but natively supports infinite streams via the `repeat` function. Also, all minibatches have the same size (note how we 'wrap around' the dataset).


In [16]:
N_train = 5
D = 4            
np.random.seed(0)
X = np.random.randn(N_train, D)
y = np.random.randn(N_train)
print(y)
batch_size = 2
dataset = tf.data.Dataset.from_tensor_slices({"X": X, "y": y})
batches = dataset.repeat().batch(batch_size)

print('batchified version')
step = 0
num_minibatches = 4
for batch in batches:
    if step >= num_minibatches:
        break
    # print(type(batch["X"])) #<class 'tensorflow.python.framework.ops.EagerTensor'
    x, y = batch["X"].numpy(), batch["y"].numpy()
    print(y)
    step = step + 1


print('batchified version v2')
batch_stream = batches.as_numpy_iterator()
for step in range(num_minibatches):
  batch = batch_stream.next()
  # print(type(batch["X"])) #<class 'numpy.ndarray'>
  x, y = batch["X"], batch["y"]
  print(y)
  step = step + 1

[-2.55298982  0.6536186   0.8644362  -0.74216502  2.26975462]
batchified version
[-2.55298982  0.6536186 ]
[ 0.8644362  -0.74216502]
[ 2.26975462 -2.55298982]
[0.6536186 0.8644362]
batchified version v2
[-2.55298982  0.6536186 ]
[ 0.8644362  -0.74216502]
[ 2.26975462 -2.55298982]
[0.6536186 0.8644362]



TF also has preprocessed datasets, available from
https://www.tensorflow.org/datasets

In [19]:
import tensorflow_datasets as tfds
dataset = tfds.load(name="mnist", split=tfds.Split.TRAIN)

batches = dataset.repeat().batch(batch_size)

step = 0
for batch in batches:
    if step >= num_minibatches:
        break
    X, y = batch['image'], batch['label']
    print(type(X))
    print(X.shape)
    step = step + 1

<class 'tensorflow.python.framework.ops.EagerTensor'>
(2, 28, 28, 1)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(2, 28, 28, 1)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(2, 28, 28, 1)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(2, 28, 28, 1)


# Autodiff

We use binary logistic regression as an example

In [26]:
## Compute gradient of loss "by hand" using numpy

from scipy.special import logsumexp

def BCE_with_logits(logits, targets):
    N = logits.shape[0]
    logits = logits.reshape(N,1)
    logits_plus = np.hstack([np.zeros((N,1)), logits]) # e^0=1
    logits_minus = np.hstack([np.zeros((N,1)), -logits])
    logp1 = -logsumexp(logits_minus, axis=1)
    logp0 = -logsumexp(logits_plus, axis=1)
    logprobs = logp1 * targets + logp0 * (1-targets)
    return -np.sum(logprobs)/N

def sigmoid(x): return 0.5 * (np.tanh(x / 2.) + 1)

def predict_logit(weights, inputs):
    return np.dot(inputs, weights) # Already vectorized

def predict_prob(weights, inputs):
    return sigmoid(predict_logit(weights, inputs))

def NLL(weights, batch):
    X, y = batch
    logits = predict_logit(weights, X)
    return BCE_with_logits(logits, y)

def NLL_grad(weights, batch):
    X, y = batch
    N = X.shape[0]
    mu = predict_prob(weights, X)
    g = np.sum(np.dot(np.diag(mu - y), X), axis=0)/N
    return g

np.random.seed(0)
N = 100
D = 5
X = np.random.randn(N, D)
w = 10*np.random.randn(D)
mu = predict_prob(w, X)
y = np.random.binomial(n=1, p=mu, size=N)

X_test = X
y_test = y

y_pred = predict_prob(w, X_test)
loss = NLL(w, (X_test, y_test))
grad_np = NLL_grad(w, (X_test, y_test))
print("params {}".format(w))
#print("pred {}".format(y_pred))
print("loss {}".format(loss))
print("grad {}".format(grad_np))

params [ 3.8273243  -0.34242281 10.96346846 -2.34215801 -3.47450652]
loss 0.05501843790657687
grad [-0.01360904  0.00325892  0.00844617  0.00848175  0.01390088]


In [27]:

w_tf = tf.Variable(np.reshape(w, (D,1)))  
x_test_tf = tf.convert_to_tensor(X_test, dtype=np.float64) 
y_test_tf = tf.convert_to_tensor(np.reshape(y_test, (-1,1)), dtype=np.float64)
with tf.GradientTape() as tape:
    logits = tf.linalg.matmul(x_test_tf, w_tf)
    y_pred = tf.math.sigmoid(logits)
    loss_batch = tf.nn.sigmoid_cross_entropy_with_logits(y_test_tf, logits)
    loss_tf = tf.reduce_mean(loss_batch, axis=0)
grad_tf = tape.gradient(loss_tf, [w_tf])
grad_tf = grad_tf[0][:,0].numpy()
assert np.allclose(grad_np, grad_tf)

print("params {}".format(w_tf))
#print("pred {}".format(y_pred))
print("loss {}".format(loss_tf))
print("grad {}".format(grad_tf))

params <tf.Variable 'Variable:0' shape=(5, 1) dtype=float64, numpy=
array([[ 3.8273243 ],
       [-0.34242281],
       [10.96346846],
       [-2.34215801],
       [-3.47450652]])>
loss [0.05501844]
grad [-0.01360904  0.00325892  0.00844617  0.00848175  0.01390088]
